<a href="https://colab.research.google.com/github/Merha23/AI_Tigrinya_Translation/blob/main/Final_Project_Source_Code_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

      **Integration GitHub with Google Colab**

In [2]:
# Clone the GitHub Repository in Google Colab
# Authenticate Google Colab with GitHub
# Clone with Authentication
# Generate a GitHub Personal Access Token (PAT)
# How to Use Your GitHub Token in Google Colab Securely
# cloning your repository using the token:
# Use the stored token to clone your GitHub repository in Google Colab:
# Clone the Repository Securely in Colab
# Instead of using your username/password, use the token as follows:

import os
from getpass import getpass

token = getpass('Enter your GitHub Personal Access Token: ')
os.environ["GITHUB_TOKEN"] = token

repo_url = f"https://{token}@github.com/Merha23/AI_Tigrinya_Translation.git"
!git clone {repo_url}
%cd AI_Tigrinya_Translation


Enter your GitHub Personal Access Token: ··········
Cloning into 'AI_Tigrinya_Translation'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 77 (delta 43), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 91.00 KiB | 3.79 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/AI_Tigrinya_Translation/AI_Tigrinya_Translation


In [3]:
# Run this python code and if cloning is successful, it should display the repository contents.

!ls


 Cheat_Sheet_Python_Code.ipynb		   LICENSE		      Practices.ipynb
 Final_Project_Source_Code_Program.ipynb  'Medical Translation.csv'   README.md


In [9]:
# Mount Google Drive in Colab
# Run this command in Colab to access your Drive:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# Pull the Latest Changes from GitHub
# Before pushing your changes, you need to sync your local copy of the repository with the remote one.
# Pull the latest changes:
# Run the following command to fetch and merge the latest changes from GitHub into your local branch:

!git pull https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

From https://github.com/Merha23/AI_Tigrinya_Translation
 * branch            main       -> FETCH_HEAD
Already up to date.


In [11]:
# Push Your Changes to GitHub
# After successfully pulling the latest changes and resolving any conflicts (if needed), you can now push your local changes:

!git push https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

Everything up-to-date


In [11]:
# To reset to the remote state (undo local changes):

!git reset --hard origin/main


HEAD is now at 6c444c4 Created using Colab


In [2]:
# To fetch the latest changes without merging them:

!git fetch


fatal: not a git repository (or any of the parent directories): .git


In [6]:
# Push Updates from Colab to GitHub
# After editing files, commit and push:

!git config --global user.email "halefomhailemariam19@gmail.com"
!git config --global user.name "Halefom Hailemariam"
!git add .
!git commit -m "Updated final project source code program"
!git push https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


    **Load the CSV File into a Pandas DataFrame**

In [4]:
import pandas as pd

df = pd.read_csv("Medical Translation.csv", encoding='utf-8')
df.head()  # Display first few rows

,id,english,tigrinya,domain
0,1,Information for parents,ንወለዲ ዝኸውን ሓበሬታ,medical
1,2,"If you choose not to vaccinate your child, und...",ውላድኩም ከይተኽትብዎ እንተመሪጽኩም፣ ሓደጋታትን ሓላፍነትን ተረድኡ,medical
2,3,Telling healthcare professionals your child’s ...,ንሰብ ሞያ ክንክን ጥዕና ኩነታት ክታበት ውላድኩም ምሕባር ብኽልተ ምኽንያ...,medical
3,4,If you choose to delay some vaccines or refuse...,ንገሊኡ ክታበታት ክተደናጉዩ ወይ ንገሊኡ ክታበታት ምሉእ ብምሉእ ክትነጽጉ...,medical
4,5,Please follow these steps to protect your chil...,ንውላድኩም፣ ንስድራቤትኩምን ንኻልኦትን ንምክልኻል በጃኹም ነዞም ስጉምትታ...,medical


     **Data Cleaning & Handling Missing Values**

     Before tokenization, we need to remove any unnecessary data such as missing values, duplicates, or improperly formatted sentences.

In [5]:
# Remove empty or missing values
df = df.dropna()

# Remove duplicates
df = df.drop_duplicates()

# Strip unwanted whitespace
df['english'] = df['english'].str.strip()
df['tigrinya'] = df['tigrinya'].str.strip()

print(f"Dataset size after cleaning: {df.shape}")

Dataset size after cleaning: (727, 4)


                **Sentence Tokenization**
  Since we are training a sequence-to-sequence model, we must tokenize each sentence.

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Ensure column names match exactly
df['tokenized_english'] = df['english'].apply(lambda x: str(x).split('. '))
df['tokenized_tigrinya'] = df['tigrinya'].apply(lambda x: str(x).split('። '))
df[['tokenized_english', 'tokenized_tigrinya']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,tokenized_english,tokenized_tigrinya
0,[Information for parents],[ንወለዲ ዝኸውን ሓበሬታ]
1,"[If you choose not to vaccinate your child, un...",[ውላድኩም ከይተኽትብዎ እንተመሪጽኩም፣ ሓደጋታትን ሓላፍነትን ተረድኡ]
2,[Telling healthcare professionals your child’s...,[ንሰብ ሞያ ክንክን ጥዕና ኩነታት ክታበት ውላድኩም ምሕባር ብኽልተ ምኽን...
3,[If you choose to delay some vaccines or refus...,[ንገሊኡ ክታበታት ክተደናጉዩ ወይ ንገሊኡ ክታበታት ምሉእ ብምሉእ ክትነጽ...
4,[Please follow these steps to protect your chi...,[ንውላድኩም፣ ንስድራቤትኩምን ንኻልኦትን ንምክልኻል በጃኹም ነዞም ስጉምት...


                **Check Sentence Length Distribution**

We need to analyze the length of sentences to ensure they fit within the model's input constraints.

In [19]:
# Check sentence length distribution
df['eng_length'] = df['english'].apply(lambda x: len(str(x).split()))
df['tir_length'] = df['tigrinya'].apply(lambda x: len(str(x).split()))

df[['eng_length', 'tir_length']].describe()

,eng_length,tir_length
count,727.000000,727.000000
mean,10.493810,9.477304
std,10.393082,8.752440
min,1.000000,1.000000
25%,2.000000,3.000000
50%,7.000000,7.000000
75%,16.000000,14.000000
max,69.000000,62.000000


            **Filtering Extremely Long or Short Sentences**

Very short or long sentences might reduce model performance. We filter out sentences that are too short (<3 words) or too long (>128 words).

In [27]:
# Filter out sentences that are too short or too long
df = df[(df['eng_length'] >= 3) & (df['eng_length'] <= 128)]
df = df[(df['tir_length'] >= 3) &  (df['tir_length'] <= 128)]

print(f"Dataset size after filtering: {df.shape}")

Dataset size after filtering: (538, 8)


               **Save the Processed Dataset**

After preprocessing, we save the cleaned and tokenized dataset for use in model training.

In [38]:
df.to_csv("cleaned_Medical Translation.csv", index=False, encoding='utf-8')
print("Dataset saved successfully!")

Dataset saved successfully!


In [36]:
import os
print(os.getcwd())  # Shows current directory
print(os.listdir())  # Lists all files in the directory


/content/AI_Tigrinya_Translation/AI_Tigrinya_Translation
['Final_Project_Source_Code_Program.ipynb', 'Cleaned_Medical Translation.csv', '.git', 'cleaned_Medical Translation.csv', 'README.md', 'LICENSE', '.gitignore', 'Cleaned Medical Translation.csv', 'Practices.ipynb', 'Medical Translation.csv', 'Cheat_Sheet_Python_Code.ipynb']


In [37]:
from google.colab import files
files.download("cleaned_Medical Translation.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(df.sample(5))  # To heck for corrupted text (especially Tigrinya encoding issues)


      id                                            english  \
29    30                 You cannot tell who is contagious.   
285  286  Payment paid directly to the City of Seattle/SPP.   
2      3  Telling healthcare professionals your child’s ...   
658  659                                              Wages   
400  401  Answer the food situation and demographic ques...   

                                              tigrinya   domain  
29                      መን ተላባዒ ምዃኑ ክትሕብሩ ኣይትኽእሉን ኢኹም።  medical  
285                ክፍሊት ብቐጥታ ናብ ከተማ Seattle /SPP ዝኽፈል።    legal  
2    ንሰብ ሞያ ክንክን ጥዕና ኩነታት ክታበት ውላድኩም ምሕባር ብኽልተ ምኽንያ...  medical  
658                                                ደሞዝ    legal  
400                        ኩነታት መግብን ዲሞግራፊካዊ ሕቶታትን መልሱ    legal  


In [ ]:
df = pd.read_csv("Medical Translation.csv", encoding="utf-8")  # Try different encodings if needed


In [ ]:
print(df.isnull().sum())  # Identify missing data
df = df.dropna()  # Remove missing rows if necessary


id           0
english     20
tigrinya    18
domain       0
dtype: int64


In [ ]:
print(df.columns)  # Verify column names


Index(['id', 'english', 'tigrinya', 'domain'], dtype='object')


           **Tokenization & Data Formatting for Fine-Tuning since we are used NLLB-200 model**

In [ ]:
!pip install transformers sentencepiece  #To install the Hugging Face Transformers library


In [ ]:
from transformers import AutoTokenizer             #To load the tokenizer

model_name = "facebook/nllb-200-distilled-600M"  # Or another variant
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
def tokenize_function(examples):           #To tokenize the dataset
    return tokenizer(examples["english"], padding="max_length", truncation=True)

df["tokenized"] = df["english"].apply(lambda x: tokenizer(x, return_tensors="pt")["input_ids"])


             **Prepare Data for Fine-Tuning**

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.8 MB/s eta 0:00:00


In [ ]:
train_dataset = train_dataset.filter(lambda x: x["english"] is not None and x["tigrinya"] is not None)
val_dataset = val_dataset.filter(lambda x: x["english"] is not None and x["tigrinya"] is not None)


Filter:   0%|          | 0/654 [00:00<?, ? examples/s]

Filter:   0%|          | 0/73 [00:00<?, ? examples/s]

In [ ]:
def replace_missing(example):
    if example["english"] is None or example["english"] == "":
        example["english"] = ""
    if example["tigrinya"] is None or example["tigrinya"] == "":
        example["tigrinya"] = ""
    return example

train_dataset = train_dataset.map(replace_missing)
val_dataset = val_dataset.map(replace_missing)


Map:   0%|          | 0/654 [00:00<?, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer # To store tokenized values as Python lists instead of PyTorch tensors

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Convert tokenized tensors into lists
df["tokenized"] = df["english"].apply(lambda x: tokenizer(x, return_tensors="pt")["input_ids"].tolist() if isinstance(x, str) else None)

# Check if conversion is successful
print(df["tokenized"].head())


0                     [[256047, 82090, 351, 45144, 2]]
1    [[256047, 9654, 1259, 93809, 2294, 202, 125691...
2    [[256047, 109, 54621, 37947, 22445, 113448, 42...
3    [[256047, 9654, 1259, 93809, 202, 202374, 9949...
4    [[256047, 62470, 25009, 15807, 138014, 202, 97...
Name: tokenized, dtype: object


In [ ]:
from datasets import Dataset    # To Convert DataFrame into Hugging Face Dataset format

dataset = Dataset.from_pandas(df)


In [ ]:
from sklearn.model_selection import train_test_split # To Split into training and validation sets

train_data, val_data = train_test_split(df, test_size=0.1, random_state=42)


       **Fine-Tuning the Model**

In [ ]:
from datasets import Dataset
# Convert Pandas DataFrames into Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Remove unnecessary Pandas index column
train_dataset = train_dataset.remove_columns(["__index_level_0__"]) if "__index_level_0__" in train_dataset.column_names else train_dataset
val_dataset = val_dataset.remove_columns(["__index_level_0__"]) if "__index_level_0__" in val_dataset.column_names else val_dataset


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-600M"  # Use the correct NLLB model variant

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
